In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gc
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [3]:
from m5.functions import *
%aimport m5.functions

In [4]:
ts_sales = pd.read_pickle('../ts_sales.pk')
ts_sales['d'] = ts_sales['d'].transform(lambda x: int(x[2:]))

In [5]:
prices = pd.read_pickle('../ts_prices.pk')
prices = prices.set_index(ts_sales['date'])
prices['d'] = ts_sales['d']

In [5]:
# чаще встречающейся
for cm in prices.columns:
    cv = prices.groupby(cm)[cm].count()  
    for i, v in cv.items():                    
         if i > 0:
            prices[cm] = prices[cm].replace({0:i})
            break

In [ ]:
prices = prices.replace({0:np.nan})
# replace mean

means = prices.mean()
prices = prices.fillna(0)
for cm in prices.columns:
    prices[cm].replace({0:means[cm]})
prices = prices.fillna(0)

In [9]:
calendar_columns =['d', 'wday', 'month', 'event_name_1_ord', 'event_name_2_ord', 'date']
lag_columns = list(filter(lambda x: 'HOBBIES' in x, ts_sales.columns))
stores = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
rollings = [3, 5, 7]
lags = [3, 5]

In [10]:
agg_columns = add_aggregated_columns(ts_sales, [['FOODS', i] for i in stores])
df = prepare_ds(calendar_columns, ts_sales, lag_columns, agg_columns,
                rollings, lags)
df.to_pickle('../learn_step.pk')

3
5
28
3
5


In [11]:
df = convert_ds_to_learn(df, prices, calendar_columns, lag_columns, 
                          agg_columns, rollings, lags)
le = LabelEncoder()
le.fit(df.id.unique())
df['iid'] = le.transform(df.id)
df.to_pickle('../learn.pk')

3
5
28
3
5


In [33]:
tr_lags = [3, 5]
tr_rollings = [3, 5]
feature_names  = ['wday', 'iid'] + \
    ['sma' + str(i) for i in tr_rollings] + \
    ['sma' + str(i) + '_shop' for i in tr_rollings] + ['lag' + str(i) for i in tr_lags] + \
    ['sma' + str(i) + '_lag' + str(y) for i in tr_rollings for y in tr_lags]
categorical_features = ['event_name_1_ord']
tr_cm = feature_names + categorical_features

In [35]:
# df = pd.read_pickle('../learn.pk')
START = 500
Xy = df[(df.d > START) & (df.d < 1969 - 56)]
X = Xy[tr_cm]
y = Xy['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
Xy.to_pickle('../Xy.pk')
X_test.to_pickle('../X_test.pk')
y_test.to_pickle('../y_test.pk')
os.remove('../train.bin')
os.remove('../test.bin')
train = lgb.Dataset(X_train, label=y_train, feature_name=tr_cm, 
            categorical_feature=categorical_features).save_binary('../train.bin')
lgb.Dataset(X_test, label=y_test, reference=train).save_binary('../test.bin')